# Convolutional Layers

In [ ]:

import sys

assert sys.version_info >= (3, 7)

In [ ]:
import tensorflow as tf
from sklearn.datasets import load_sample_images
import matplotlib.pyplot as plt

In [ ]:
if tf.config.list_physical_devices('GPU'):
  print("Connected to GPU successfully!")
else:
  print("No GPU connnected")

In [ ]:
images=load_sample_images()["images"]
images=tf.keras.layers.CenterCrop(height=75,width=115)(images)
images=tf.keras.layers.Rescaling(scale=1/255)(images)

In [ ]:
images.shape

In [ ]:
plt.figure(figsize=(4,4))
for img_idx in (0,1):
    for j in (0,1):
        plt.subplot(2,2,img_idx*2+j+1)
        plt.imshow(images[img_idx,:,:,:])
        plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
tf.keras.utils.set_random_seed(42)

conv_layer=tf.keras.layers.Conv2D(filters=32,kernel_size=8)
fmap=conv_layer(images)

In [ ]:
fmap.shape

In [ ]:
for img_idx in (0,1):
    for fmap_idx in (0,1):
        plt.subplot(2,2,img_idx*2+fmap_idx+1)
        plt.imshow(fmap[img_idx,:,:,fmap_idx])
        plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
conv_layer=tf.keras.layers.Conv2D(filters=32,kernel_size=7,padding="same")
fmap=conv_layer(images)

In [ ]:
fmap.shape

In [ ]:
conv_layer=tf.keras.layers.Conv2D(filters=32,kernel_size=7,padding="same",
                                 strides=2)
fmap=conv_layer(images)
fmap.shape

In [ ]:
kernel,biases=conv_layer.get_weights()

In [ ]:
kernel.shape

In [ ]:
biases.shape

In [ ]:
filters=tf.random.normal([7,7,3,2])
biases=tf.zeros([2])
fmaps=tf.nn.conv2d(images,filters,strides=1,padding="SAME")+biases

In [ ]:
fmaps.shape

In [ ]:
import numpy as np

filters=np.zeros([7,7,3,2])
filters[:,3,:,0] = 1
filters[3,:,:,1] = 1
fmaps=tf.nn.conv2d(images,filters,strides=1,padding="SAME")+biases

for image_idx in (0,1):
  for fmap_idx in (0,1):
    plt.subplot(2,2,image_idx*2+fmap_idx+1)
    plt.imshow(fmaps[image_idx,:,:,fmap_idx],cmap="gray")
    plt.axis("off")
plt.tight_layout()

# Pooling Layers

### Max pooling

In [ ]:
max_pool=tf.keras.layers.MaxPool2D(pool_size=2)

In [ ]:
output=max_pool(images)

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(images[0])
plt.axis("off")
plt.subplot(1,2,2)
plt.imshow(output[0])
plt.axis("off")
plt.show()


##### Depth-wise pooling

In [ ]:
class DepthPool(tf.keras.layers.Layer):
  def __init__(self,pool_size=2,**kwargs):
    super().__init__(**kwargs)
    self.pool_size=pool_size

  def call(self,inputs):
    shape=tf.shape(inputs)
    groups=shape[-1]//self.pool_size
    new_shape=tf.concat([shape[:-1],[groups,self.pool_size]],axis=0)
    return tf.reduce_max(tf.reshape(inputs,new_shape),axis=-1)    #So for every [batch, height, width, group], it


In [ ]:
depth_output=DepthPool(pool_size=3)(images)

plt.figure(figsize=(10,10))
plt.subplot(1,2,1)

plt.imshow(images[0])
plt.axis("off")
plt.subplot(1,2,2)
plt.imshow(depth_output[0])
plt.axis("off")
plt.show()

##### Global Average Pooling

In [ ]:
global_avg_pooling=tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
global_avg_pooling=tf.keras.layers.Lambda(lambda X:tf.reduce_mean(X,axis=[1,2]))

# CNN Architectures

##### Tackling Fashion MNIST With a CNN

In [ ]:
mnist=tf.keras.datasets.fashion_mnist.load_data()
(X_train_full,y_train_full),(X_test,y_test)=mnist
X_train_full=np.expand_dims(X_train_full,axis=-1).astype(np.float32)/255
X_test=np.expand_dims(X_test,axis=-1).astype(np.float32)/255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

In [ ]:
from functools import partial

DefaultConv2D=partial(tf.keras.layers.Conv2D,kernel_size=3,padding="same",activation="relu",
                      kernel_initializer="he_normal")

model=tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=[28,28,1]),
    DefaultConv2D(filters=64,kernel_size=7),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation="relu",kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64,activation="relu",kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10,activation="softmax")


])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",
              metrics=["accuracy"])
model.fit(X_train,y_train,
          epochs=10,
          validation_data=(X_valid,y_valid))
score=model.evaluate(X_test,y_test)

## GoogLeNet

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Input, Concatenate
from tensorflow.keras.models import Model

def InceptionModule(x,f1,f3r,f3,f5r,f5,proj):

  path1=Conv2D(f1,kernel_size=1,padding="same",activation="relu")(x)

  path2=Conv2D(f3r,kernel_size=1,padding="same",activation="relu")(x)
  path2=Conv2D(f3,kernel_size=3,padding="same",activation="relu")(path2)

  path3=Conv2D(f5r,kernel_size=1,padding="same",activation="relu")(x)
  path3=Conv2D(f5,kernel_size=5,padding="same",activation="relu")(path3)

  path4=MaxPooling2D(pool_size=3,strides=1,padding="same")(x)
  path4=Conv2D(proj,kernel_size=1,padding="same",activation="relu")(path4)

  return Concatenate(axis=-1)([path1,path2,path3,path4])

In [ ]:
from tensorflow.keras.layers import BatchNormalization,Dense,GlobalAveragePooling2D,Activation,Input,Dropout

input_layer=Input(shape=(28,28,1))

x=Conv2D(filters=64,kernel_size=7,strides=2,padding="same",activation="relu")(input_layer)
x=MaxPooling2D(pool_size=3,strides=2,padding="same")(x)
x=BatchNormalization()(x)

x=Conv2D(64,kernel_size=1,padding="same",activation="relu")(x)
x=Conv2D(192,kernel_size=3,padding="same",activation="relu")(x)
x=BatchNormalization()(x)
x=Activation("relu")(x)
x=MaxPooling2D(pool_size=3,strides=2,padding="same")(x)

x=InceptionModule(x,64,96,128,12,32,32)
x=InceptionModule(x,128,128,192,32,96,64)
x=MaxPooling2D(pool_size=3,strides=2,padding="same")(x)

x=InceptionModule(x,192, 96, 208, 16, 48, 64)
x=InceptionModule(x,160, 112, 224, 24, 64, 64)
x=InceptionModule(x, 128, 128, 256, 24, 64, 64)
x=InceptionModule(x,112, 144, 288, 32, 64, 64)
x=InceptionModule(x,256, 160, 320, 32, 128, 128)
x=MaxPooling2D(pool_size=3,strides=2,padding="same")(x)

x=InceptionModule(x,256, 160, 320, 32, 128, 128)
x=InceptionModule(x,384, 192, 384, 48, 128, 128)
x=GlobalAveragePooling2D()(x)
x=Dropout(0.2)(x)
x=Dense(10,activation="softmax")(x)


model=Model(input_layer,x)

## ResNet-34

In [ ]:
DefaultConv2D=partial(tf.keras.layers.Conv2D,kernel_size=3,strides=1,padding="same",
                      kernel_initializer="he_normal",use_bias=False)

class ResidualUnit(tf.keras.layers.Layer):
  def __init__(self,filters,strides=1,activation="relu",**kwargs):
     super().__init__(**kwargs)
     self.activation=tf.keras.activations.get(activation)
     self.main_layers=[
         DefaultConv2D(filters,strides=strides),
         tf.keras.layers.BatchNormalization(),
         self.activation,
         DefaultConv2D(filters),
         tf.keras.layers.BatchNormalization()
     ]
     self.skip_layers=[]
     if strides>1:
      self.skip_layers=[
          DefaultConv2D(filters,kernel_size=1,strides=strides),
          tf.keras.layers.BatchNormalization()
      ]

  def call(self,inputs):
    Z=inputs
    for layer in self.main_layers:
      Z=layer(Z)
    skip_Z=inputs
    for layer in self.skip_layers:
      skip_Z=layer(skip_Z)

    return self.activation(Z+skip_Z)

In [ ]:
mean=np.mean(X_train_full,axis=(0,1,2))
std=np.std(X_train_full,axis=(0,1,2))

data_preprocessing=tf.keras.Sequential([
    tf.keras.layers.Normalization(mean=mean.tolist(),variance=(std**2).tolist()),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1)
])

In [ ]:
model=tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=[32,32,3]),
    data_preprocessing,
    DefaultConv2D(64,kernel_size=3,strides=1,padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPooling2D(pool_size=3,strides=2),
])

prev_filter=64
for filters in [64]*3 +[128]*4 + [256]*6 + [512]*3:
  strides = 1 if filters==prev_filter else 2
  model.add(ResidualUnit(filters=filters,strides=strides))
  prev_filter=filters

model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(100,activation="softmax"))


## Using Pretrained Modes from Keras

In [ ]:
model=tf.keras.applications.ResNet50(weights="imagenet")

In [ ]:
images=tf.constant(load_sample_images()["images"])
images_resized=tf.keras.layers.Resizing(height=224,width=224,crop_to_aspect_ratio=True)(images)

In [ ]:
images.shape

In [ ]:
images_resized.shape

In [ ]:
images_resized.shape

In [ ]:
inputs=tf.keras.applications.resnet50.preprocess_input(images_resized)
y_proba=model.predict(inputs)

In [ ]:
y_proba.shape

In [ ]:
top_K=tf.keras.applications.resnet50.decode_predictions(y_proba,top=5)

for idx in range(len(images)):
  print(f"Image #{idx}")
  for class_id,name,y_proba in top_K[idx]:
    print(f"{class_id}-{name:12s} {y_proba:.2%}")

# Pretrained Models for Transfer Learning

In [ ]:
import tensorflow_datasets as tfds

dataset,info=tfds.load("tf_flowers",as_supervised=True,with_info=True)
dataset_size=info.splits["train"].num_examples
class_names=info.features["label"].names
num_classes=info.features["label"].num_classes

In [ ]:

test_set_raw,valid_set_raw,train_set_raw=tfds.load(
    "tf_flowers",
    split=["train[:10%]","train[10%:25%]","train[25%:]"],
    as_supervised=True,

    )

In [ ]:
class_names

In [ ]:
dataset_size

In [ ]:
num_classes

In [ ]:
plt.figure(figsize=(7,7))
idx=0
for image,label in train_set_raw.take(9):
  idx+=1
  plt.subplot(3,3,idx)
  plt.imshow(image)
  plt.title(f"Class:{class_names[label]}")
  plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
batch_size=32
preprocess=tf.keras.Sequential([
    tf.keras.layers.Resizing(height=224,width=224,crop_to_aspect_ratio=True),
    tf.keras.layers.Lambda(tf.keras.applications.xception.preprocess_input)
])
train_set=train_set_raw.map(lambda X,y: (preprocess(X),y))
train_set=train_set.shuffle(1000,seed=42).batch(batch_size).prefetch(1)
valid_set=valid_set_raw.map(lambda X,y:(preprocess(X),y)).batch(batch_size)
test_set=test_set_raw.map(lambda X,y:(preprocess(X),y)).batch(batch_size)

In [ ]:
plt.figure(figsize=(7,7))
for X_batch, y_batch in train_set.take(1):
    for index in range(9):
        plt.subplot(3, 3, index + 1)
        plt.imshow((X_batch[index] + 1) / 2)  # rescale to 0–1 for imshow()
        plt.title(f"Class: {class_names[y_batch[index]]}")
        plt.axis("off")

plt.show()

In [ ]:
data_agumentation=tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal",seed=42),
    tf.keras.layers.RandomRotation(0.05,seed=42),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1)
])


In [ ]:
plt.figure(figsize=(7,7))
for X_batch,y_batch in train_set.take(1):
  X_batch_agumentation=data_agumentation(X_batch,training=True)
  for idx in range(9):
    plt.subplot(3,3,idx+1)
    plt.imshow(np.clip((X_batch_agumentation[idx]+1)/2,0,1))
    plt.axis("off")
plt.show()

In [ ]:
tf.random.set_seed(42)

base_model=tf.keras.applications.xception.Xception(weights="imagenet",
                                                   include_top=False)
avg=tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output=tf.keras.layers.Dense(num_classes,activation="softmax")(avg)
model=tf.keras.Model(inputs=base_model.inputs,outputs=output)

In [ ]:
for layer in base_model.layers:
  layer.trainable=False

In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,
              metrics=["accuracy"])
history=model.fit(train_set,
                  epochs=3,
                  validation_data=(valid_set))

In [ ]:
for indices in zip(range(33),range(33,66),range(66,99),range(99,132)):
  for idx in indices:
    print(f"{idx:3}: {base_model.layers[idx].name:22}",end="")
  print()

In [ ]:
for layer in base_model.layers[56:]:
  layer.trainable=True

In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,
              metrics=["accuracy"])

history=model.fit(train_set,
                  epochs=10,
                  validation_data=(valid_set))

# Exercise

### Build your own CNN from scratch and try to achieve the highest possible accuracy on MNIST.

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
mnist=tf.keras.datasets.mnist.load_data()

In [ ]:
(X_train_full,y_train_full),(X_test,y_test)=mnist
X_train_full=X_train_full/255.
X_test_full=X_test/255.

X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]


In [ ]:
X_train.shape

In [ ]:
X_train=X_train[...,np.newaxis]
X_valid=X_valid[...,np.newaxis]
X_test=X_test[...,np.newaxis]

In [ ]:
X_train.shape

In [ ]:
tf.keras.utils.set_random_seed(42)

model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,kernel_size=3,padding="same",activation="relu",
                           kernel_initializer="he_normal"),
    tf.keras.layers.Conv2D(64,kernel_size=3,padding="same",activation="relu",
                           kernel_initializer="he_normal"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(128,activation="relu",kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10,activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",
              metrics=["accuracy"])
model.fit(X_train,y_train,
          epochs=10,
          validation_data=(X_valid,y_valid))
model.evaluate(X_test,y_test)

## Satellite Image Classifier

In [ ]:
import tensorflow_datasets as tfds

(ds_train,ds_valid,ds_test),info=tfds.load(
    "eurosat",
    split=["train[:70%]","train[70%:85%]","train[85%:]"],
    as_supervised=True,
    with_info=True
)

In [ ]:
dataset_size=info.splits["train"].num_examples
class_names=info.features["label"].names
num_classes=info.features["label"].num_classes

In [ ]:
dataset_size

In [ ]:
class_names

In [ ]:
import matplotlib.pyplot as plt
index=0
for image,label in ds_train.take(9):
  index+=1
  plt.subplot(3,3,index)
  plt.imshow(image)
  plt.title(class_names[label])
  plt.axis("off")


In [ ]:
image.shape

In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()

batch_size=32

preprocess=tf.keras.Sequential([
    tf.keras.layers.Resizing(height=64,width=64,crop_to_aspect_ratio=True),
    tf.keras.layers.Lambda(tf.keras.applications.mobilenet_v2.preprocess_input)
])

data_agumentation=tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomRotation(0.1)
])

train_set=ds_train.map(lambda X,y:(preprocess(X),y))
train_set=train_set.shuffle(1000).batch(batch_size).prefetch(1)
valid_set=ds_valid.map(lambda X,y:(preprocess(X),y)).batch(batch_size)
test_set=ds_test.map(lambda X,y:(preprocess(X),y)).batch(batch_size)


In [ ]:
base_model=tf.keras.applications.MobileNetV2(
    input_shape=(64,64,3),
    weights="imagenet",
    include_top=False
)

avg_layer=tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x=tf.keras.layers.Dropout(0.3)(avg_layer)
output=tf.keras.layers.Dense(num_classes,activation="softmax")(x)
model=tf.keras.Model(inputs=base_model.input,outputs=output)

In [ ]:
for layer in base_model.layers:
  layer.trainable=False

optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,
              metrics=["accuracy"])

model.fit(train_set,
          epochs=4,
          validation_data=(valid_set))

In [ ]:
len(base_model.layers)

In [ ]:
for indices in zip(range(39), range(39, 78), range(78, 117), range(117, 154)):
    for idx in indices:
        print(f"{idx:3}: {base_model.layers[idx].name:33}", end="")
    print()

In [ ]:
for layer in base_model.layers[72:]:
  layer.trainable=True

optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,
              metrics=["accuracy"])

history=model.fit(train_set,
          epochs=10,
          validation_data=(valid_set))

In [ ]:
model.evaluate(test_set)